In [1]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkConf, SparkContext
import json
import time
import math 
import sys, math
from pyspark.sql.functions import col
import pandas as pd
from pyspark.ml.stat import Correlation

In [ ]:
# WIRD NICHT BENÖTIGT
#conf = SparkConf().setAppName("Korrelation").setMaster("local[*]").set("spark.executor.memory", '10g').set("spark.driver.memory",'10g')
#conf = SparkConf().setAppName("Korrelation_new").setMaster("spark://192.168.1.247:7077")
#sc = SparkContext(conf=conf)

bei .master ("local[*]") einsetzen, oder ein master + slaves starten (steigert Performance um das 4-fache)

In [2]:
spark = pyspark.sql.SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Amazon Reviews") \
    .getOrCreate()
# .master("spark://192.168.1.247:7077") \

# Lade JSON Datei
df = spark.read.json(r'C:\Users\klass\data\Appliances.json')

# Entferne Spalten
df = df.drop('verified', 'reviewTime',"reviewerID", "asin","reviewerName","summary","unixReviewTime","vote", "style", "image")
df = df.fillna(" ", subset=["reviewText"])
df = df.withColumn("Textlänge_integer", pyspark.sql.functions.length('reviewText'))
df = df.withColumn("Textlänge", col("Textlänge_integer").cast("double"))
df = df.drop("reviewText", "Textlänge_integer")
df.fillna(0, subset=["Textlänge"])

# Dataframe in RDD konvertieren.
partitionen = 10
data = df.rdd.repartition(partitionen)

# Korrelation step by step selber berechnet

In [3]:
def calculate_mean(data, column):
    return data.map(lambda x: x[column]).sum() / n

def calculate_std(data, column, mean):
    return math.sqrt(data.map(lambda x: (x[column] - mean)**2).sum() / n)

def calculate_cov(data, column1, column2, mean1, mean2):
    return data.map(lambda x: (x[column1] - mean1) * (x[column2] - mean2)).sum() / n

start_time = time.time()
n = data.count()
mean_col1 = calculate_mean(data, 0)
mean_col2 = calculate_mean(data, 1)
std_col1 = calculate_std(data, 0, mean_col1)
std_col2 = calculate_std(data, 1, mean_col2)
covXY = calculate_cov(data, 0, 1, mean_col1, mean_col2)
correlation = covXY / (std_col1 * std_col2)

print("Korrelation: ", correlation)
print(f"Execution time: {time.time() - start_time}")

Korrelation:  -0.17476299418809313
Execution time: 50.51594424247742


# Pyspark.SQL Korrelation

In [4]:
#Korrelation berechnen
start_time = time.time()
correlation = pyspark.sql.DataFrameStatFunctions(df).corr("overall", "Textlänge")
print("Korrelation: ", correlation)
print(f"Execution time: {time.time() - start_time}")

Korrelation:  -0.1747629941880985
Execution time: 0.5975379943847656


# Pandas Korrelation

In [5]:
#DataFrame in ein Pandas_Dataframe umwandeln
start_time = time.time()
data = df.toPandas()
print(f"Execution time: {time.time() - start_time}")

Execution time: 2.397836446762085


In [6]:
#Korrelation mit Panda-DataFrame berechnen
start_time = time.time()
print(data.corr(numeric_only = True))
print(f"Execution time: {time.time() - start_time}")

            overall  Textlänge
overall    1.000000  -0.174763
Textlänge -0.174763   1.000000
Execution time: 0.029915571212768555


# Code um Duplikate zu bilden

In [ ]:
df = spark.read.json(r'C:\Users\klass\data\all_beauty.json')

# Inhalt duplizieren
df = df.union(df)
df = df.coalesce(1)

# Inhalt in eine Datei schreiben
df.write.json("duplicated_large_file.json", mode="overwrite")